In [86]:
import tweepy
import json
import codecs

## fill in your Twitter credentials 
access_token = ''
access_token_secret = ''
consumer_key = ''
consumer_secret = ''

## Set up an instance of the REST API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Trending tweets

In [110]:
#Worldwide trending tweets
world_trending_tweets = api.trends_place(1)

us_trending_tweets = api.trends_place(23424977)

Let's find out common tweets in world and US

In [111]:
world_trends_set = set([trend['name'] 
                        for trend in world_trending_tweets[0]['trends']])

us_trends_set = set([trend['name'] 
                     for trend in us_trending_tweets[0]['trends']]) 

common_trends = world_trends_set.intersection(us_trends_set)

print(common_trends)

{'#WorldPressFreedomDay', '#ThursdayThoughts', '#OffendMeIn3Words', '#LibertadDePrensa', '#AdobeSummit'}


Seems like "WorldPressFreedomDay" is trending. Let's check out what people have to say about it.

In [112]:
#search query and store your results in a variable
search_results = api.search(q = "#WorldPressFreedomDay", lang = "en", result_type = "recent", count = 100)
search_results

[Status(_api=<tweepy.api.API object at 0x000002316A7A3B38>, _json={'created_at': 'Thu May 03 11:20:49 +0000 2018', 'id': 992001024847237120, 'id_str': '992001024847237120', 'text': 'RT @camanpour: I know journalists everywhere will continue to pick up their gear, go out, and cover the important stories -- even if it mea…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'camanpour', 'name': 'Christiane Amanpour', 'id': 69181624, 'id_str': '69181624', 'indices': [3, 13]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 97605710, 'id_str': '97605710', 'name': 'sarod', 'screen_name': 'sarod30', 'location': 'texas', 'description': '', 'url': 

In [119]:
#Experiement - Tried Creating json dump from tweepy output
a = json.dumps(search_results[0]._json['created_at'])
b = json.dumps(search_results[0]._json['retweeted'])
b

'false'

Once we know how to collect Json data from tweepy. Now we can extract details about tweets.

In [114]:
#Courtsey book - Mining the Social Web

status_texts = [ status._json['text'] 
                 for status in search_results ]

screen_names = [ user_mention['screen_name'] 
                 for status in search_results
                     for user_mention in status._json['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in search_results
                 for hashtag in status._json['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

In [115]:
print(json.dumps(status_texts[0:5], indent=1))
print(json.dumps(screen_names[0:5], indent=1))
print(json.dumps(hashtags[0:5], indent=1))
print(json.dumps(words[0:5], indent=1))

[
 "RT @camanpour: I know journalists everywhere will continue to pick up their gear, go out, and cover the important stories -- even if it mea\u2026",
 "RT @ShehuSani: #WorldPressFreedomDay the threat to press freedom used to be from the Government and its security apparatuses, but now added\u2026",
 "RT @apnnewsindia: The worst countries for #PressFreedom\n \n1 #NorthKorea \ud83c\uddf0\ud83c\uddf5 \n2 #Eritrea \ud83c\uddea\ud83c\uddf7 \n3 #Turkmenistan \ud83c\uddf9\ud83c\uddf2 \n4 #Syria \ud83c\uddf8\ud83c\uddfe \n5 #China \ud83c\udde8\ud83c\uddf3 \n6\u2026",
 "RT @Kirtanmahto: #WorldPressFreedomDay\nFor money and TRP you will fall as \nwe  never think \nreally hate you #Zee\u0939\u093f\u0902\u0926\u0941\u0938\u094d\u0924\u093e\u0928_FakeNews\nIf are\u2026",
 "RT @firstpost: On #WorldPressFreedomDay, find out how #India fares on the press freedom index. https://t.co/mFKzMwkvRx"
]
[
 "camanpour",
 "ShehuSani",
 "apnnewsindia",
 "Kirtanmahto",
 "firstpost"
]
[
 "WorldPressFreedomDay",


Creating a basic frequency distribution from the words in tweets

In [116]:
from collections import Counter

for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print(c.most_common()[:10]) # top 10
    print()

[('RT', 88), ('the', 78), ('to', 56), ('of', 53), ('#WorldPressFreedomDay', 37), ('and', 34), ('is', 33), ('in', 33), ('for', 26), ('freedom', 21)]

[('foreignoffice', 3), ('AmnestyNigeria', 3), ('pressfreedom', 3), ('narendramodi', 3), ('euronews', 3), ('dhruv_rathee', 3), ('drshraddha16', 3), ('ZeeNews', 3), ('INCIndia', 3), ('camanpour', 2)]

[('WorldPressFreedomDay', 56), ('PressFreedom', 5), ('Accra', 3), ('KathuaKiGawahi', 3), ('Nigerians', 2), ('FreeThePress', 2), ('online', 2), ('regulation', 2), ('ThursdayThoughts', 2), ('ShifaGardi', 2)]



Using prettytable to display tuples in a nice tabular format

In [117]:
from prettytable import PrettyTable

for label, data in (('Word', words), 
                    ('Screen Name', screen_names), 
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:10] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

+-----------------------+-------+
| Word                  | Count |
+-----------------------+-------+
| RT                    |    88 |
| the                   |    78 |
| to                    |    56 |
| of                    |    53 |
| #WorldPressFreedomDay |    37 |
| and                   |    34 |
| is                    |    33 |
| in                    |    33 |
| for                   |    26 |
| freedom               |    21 |
+-----------------------+-------+
+----------------+-------+
| Screen Name    | Count |
+----------------+-------+
| foreignoffice  |     3 |
| AmnestyNigeria |     3 |
| pressfreedom   |     3 |
| narendramodi   |     3 |
| euronews       |     3 |
| dhruv_rathee   |     3 |
| drshraddha16   |     3 |
| ZeeNews        |     3 |
| INCIndia       |     3 |
| camanpour      |     2 |
+----------------+-------+
+----------------------+-------+
| Hashtag              | Count |
+----------------------+-------+
| WorldPressFreedomDay |    56 |
| PressFreedom

In [126]:
retweets = [
            # Store out a tuple of these three values ...
            (status._json['retweet_count'], 
             status._json['user']['screen_name'],
             status._json['text']) 
            
            # ... for each status ...
            for status in search_results
            
            # ... so long as the status meets this condition.
                if json.dumps(status._json['retweeted']) == 'false'
                        
           ]

# Slice off the first 5 from the sorted results and display each item in the tuple

pt = PrettyTable(field_names=['Count', 'Screen Name', 'Text'])
[ pt.add_row(row) for row in sorted(retweets, reverse=True)[:5] ]
pt.max_width['Text'] = 50
pt.align= 'l'
print(pt)

+-------+---------------+----------------------------------------------------+
| Count | Screen Name   | Text                                               |
+-------+---------------+----------------------------------------------------+
| 1371  | SarooparamT   | RT @narendramodi: I would once again like to       |
|       |               | appreciate the proactive role of everyone on       |
|       |               | social media for their commendable work towards s… |
| 1168  | Arunponsingh  | RT @narendramodi: A free press makes a stronger    |
|       |               | democracy! Today on #WorldPressFreedomDay, let us  |
|       |               | reaffirm our commitment towards steadfast…         |
| 1168  | AbhayKubrey   | RT @narendramodi: A free press makes a stronger    |
|       |               | democracy! Today on #WorldPressFreedomDay, let us  |
|       |               | reaffirm our commitment towards steadfast…         |
| 746   | LesegoThinane | RT @antonioguterres: Promo